<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/NMT_S2S_Attention_Per_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Hardware spec.**

In [1]:
!cat /proc/cpuinfo
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

## **Import headers**

In [2]:
import tensorflow as tf

from sklearn.model_selection import train_test_split

import numpy as np
import unicodedata
import re
import io

In [3]:
!pip install hazm

In [4]:
from __future__ import unicode_literals
from hazm import *

## **Mount corpora**

In [5]:
file_path = '/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Parallel Corpora/Per-Eng/manythings/pes-eng/pes.txt'

## **Preprocessing**

In [6]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn') 

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip()) # lowercase
  w = re.sub(r"([?.!,¿])", r" \1 ", w) # creating a space between every word and punctuation
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w) # replacing every word with space except those defined

  w = w.strip() # remove space from beginning and end of string
  w = '<start> '+ w + ' <end>' # later model needs this to know where to start and stop

  return w

def preprocess_per_sentence(w):
  normalizer = Normalizer() # Persian normalizer instance (Hazm)
  normalizer.normalize(w)
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = '<start> '+ w + ' <end>'

  return w


## **An example**

In [7]:
eng_sentence = "You have beautiful hands."
per_sentence ="تو دستهای زیبایی داری."


print(preprocess_sentence(eng_sentence))
print(preprocess_per_sentence(per_sentence))

<start> you have beautiful hands . <end>
<start> تو دستهای زیبایی داری .  <end>


In [24]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding="UTF-8").read().strip().split('\n')
  word_pairs = [[preprocess_per_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
  return word_pairs

  #return zip(*word_pairs) #zip used for parallel iteration

In [25]:
print(create_dataset(file_path, None))

[['<start> Who ?  <end>', '<start> چه کسی؟ <end>', '<start> CC-BY 2 . 0 (France) Attribution: tatoeba . org #2083030 (CK) & #4746184 (mahdiye) <end>'], ['<start> Go on .  <end>', '<start> ادامه بده ( ادامه دادن ) <end>', '<start> CC-BY 2 . 0 (France) Attribution: tatoeba . org #2230774 (CK) & #6669169 (arashorosia) <end>'], ['<start> Smile .  <end>', '<start> لبخند بزن .  <end>', '<start> CC-BY 2 . 0 (France) Attribution: tatoeba . org #2764108 (CK) & #4746196 (mahdiye) <end>'], ['<start> Attack !  <end>', '<start> حمله !  <end>', '<start> CC-BY 2 . 0 (France) Attribution: tatoeba . org #1972610 (CK) & #4746181 (mahdiye) <end>'], ['<start> Got it !  <end>', '<start> گرفتم !  <end>', '<start> CC-BY 2 . 0 (France) Attribution: tatoeba . org #320484 (CM) & #888645 (mahdiye) <end>'], ['<start> I know .  <end>', '<start> من می دانم .  <end>', '<start> CC-BY 2 . 0 (France) Attribution: tatoeba . org #319990 (CK) & #888643 (mahdiye) <end>'], ['<start> Listen .  <end>', '<start> گوش کن .  <end

In [ ]:
print(en[-1])